In [58]:
import os.path
from arrow import get

#Google Sheets API
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

#Facebook API
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights

#Facebook Credentials
my_app_id = '1910236629370257'
my_app_secret = '34e61524a2997cecff31f4eb7c24860f'
my_access_token = 'EAAbJWZAZB7BZAEBO8oNOU2vBXWbV9EOY81spJJbZCZBYZA3xTjCZBPcnxVJAgPf7v0QGG4vsYrLNf0mYwp7u4LkFRgD1lRTiOihcWXCLzpSeBPhEgWoODWJYI4jD0jgoN1EvBO0WjpZBF6RDjlnlGMFZBMifNUQ3WKBGLIZAZBd1MTkW2jASFHFfvZCMHVpL6tD6PKC7PyxrHdo5'

graph = FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

fields_campanha = [
    AdsInsights.Field.campaign_id,
    AdsInsights.Field.campaign_name
]

fields_conjunto = [
    AdsInsights.Field.adset_id,
    AdsInsights.Field.adset_name
]

fields_ad = [
    AdsInsights.Field.ad_id,
    AdsInsights.Field.ad_name
]

fields_dados = [
  AdsInsights.Field.campaign_id,
  AdsInsights.Field.adset_id,
  AdsInsights.Field.ad_id,
  AdsInsights.Field.impressions,
  AdsInsights.Field.reach,
  AdsInsights.Field.clicks,
  AdsInsights.Field.spend,
  AdsInsights.Field.actions,
  AdsInsights.Field.video_p25_watched_actions,
  AdsInsights.Field.date_start,
  AdsInsights.Field.age_targeting,
  
]

params_campanha = {
    'date_preset': 'yesterday',
    'level': 'campaign',
}

params_conjunto = {
   'date_preset': 'yesterday',
   'level': 'adset',
}

params_ad = {
    'date_preset': 'yesterday',
    'level': 'ad',
    'breakdowns': ['age', 'gender']
}


valores = ['link_click', 'onsite_conversion.messaging_conversation_started_7d']

my_account = AdAccount('act_577560246260074')


#Google Credentials
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

SAMPLE_SPREADSHEET_ID = "1tzFGYAXPJoWbDQwTJJZH4HrARemy8ong-Yh8iVWybhk"


def cred():
  creds = None
  
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  service = build("sheets", "v4", credentials=creds)
  return service

service = cred()
SHEET = service.spreadsheets()

#Verifica os valores a planilha da aba desejada e retorna uma lista com eles
def verifica_planilha(aba):
    result = (
            SHEET.values()
                .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=aba+"!"+"A2:B")
                .execute()
            )
    
    values = result.get("values", [])
    return values

#Compara duas listas de array e retorna uma lista com os valores que não se repetem em ambos
def encontrar_ids_nao_repetidos(array1, array2):
    
    set_ids_arr1 = set(item[0] for item in array1)
    set_ids_arr2 = set(item[0] for item in array2)

    ids_exclusivos = set_ids_arr1.symmetric_difference(set_ids_arr2)

    resultado_final = [item for item in array1 + array2 if item[0] in ids_exclusivos]
    
    return resultado_final

#Retorna a ultima linha da planilha, ou seja, a que não possui dados
def ultima_linha(aba):
    values = verifica_planilha(aba)

    ultima_linha = len(values) + 2               

    return str(ultima_linha)

#Busca os insights de ID e NOME e aplica, sem repetí-los, na planilha
def id_valores(aba, fields, params):
    
    values = verifica_planilha(aba)

    insights = my_account.get_insights(fields=fields, params=params)

    dados = []

    for insight in insights:
      dados.append([insight[fields[0]],insight[fields[1]]])

    valores_nao_rep = encontrar_ids_nao_repetidos(dados, values)

    linha = ultima_linha(aba)


    result = (SHEET.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range= aba+ "!" + "A" + linha ,valueInputOption="RAW",body={'values':valores_nao_rep}).execute())


#Retorna as ações específicas de insight 
def acoes_especificas(insight, valor):
  for acao in insight.get(AdsInsights.Field.actions, []):
    if(valor == acao['action_type'] ):
          valor_acao = acao['value']
          return valor_acao
    
def videos (fields, params):
  insights = my_account.get_insights(fields=fields, params=params)
  for insight in insights:
    print(insight['age'])
       
     

#Busca os dados principais de insight e aplica na planilha
def dados_insights(aba, fields, params):
    
    linha = ultima_linha(aba)

    insights = my_account.get_insights(fields=fields, params=params)

    dados = []

    for insight in insights:
      valores_especificos = []
      for valor in valores:
        valor_acao = acoes_especificas(insight, valor)
        if (valor_acao == None):
          valor_acao = "None"
        valores_especificos.append(valor_acao)
        
      dados.append([insight[fields[0]],insight[fields[1]],insight[fields[2]], insight[fields[3]], insight[fields[4]], insight[fields[5]], insight[fields[6]], *valores_especificos, insight['age'], insight['gender'], insight[fields[9]]])

    print(dados)
    result = (SHEET.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range= aba+ "!" + "A" + linha ,valueInputOption="RAW",body={'values':dados}).execute())



#Chamando as funções a cima
if __name__ == "__main__":
  id_valores("CAMPANHA", fields_campanha, params_campanha)
  id_valores("CONJUNTO", fields_conjunto, params_conjunto)
  id_valores("ANUNCIO", fields_ad, params_ad)
  dados_insights("DADOS", fields_dados, params_ad)
  #videos(fields_dados, params_ad)
  


[['120203799059160771', '120205102953770771', '120205102953760771', '4', '4', '0', '0.02', 'None', 'None', '18-24', 'female', '2023-12-20'], ['120203799059160771', '120205102953770771', '120205102953760771', '8', '5', '0', '0.07', 'None', 'None', '18-24', 'male', '2023-12-20'], ['120203799059160771', '120205102953770771', '120205102953760771', '2', '2', '0', '0.05', 'None', 'None', '25-34', 'female', '2023-12-20'], ['120203799059160771', '120205102953770771', '120205102953760771', '2', '2', '0', '0.06', 'None', 'None', '25-34', 'male', '2023-12-20'], ['120203799059160771', '120205102953770771', '120205102953760771', '4', '3', '0', '0.09', 'None', 'None', '35-44', 'female', '2023-12-20'], ['120203799059160771', '120205102953770771', '120205102953760771', '3', '3', '0', '0.03', 'None', 'None', '35-44', 'male', '2023-12-20'], ['120203799059160771', '120205102953770771', '120205102953780771', '79', '68', '2', '1.04', 'None', 'None', '18-24', 'female', '2023-12-20'], ['120203799059160771', 